In [ ]:
import numpy as np
import pandas as pd
import os 
import seaborn as sns
import matplotlib.pyplot as plt
import torch 
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from torch.autograd.functional import hessian


from torch.func import jacfwd 
from torch.func import vmap 

from torchdiffeq import odeint
from standard_functions import LR_Scheduler, train_validation_split, plot_all

path = os.getcwd()
data_path = os.path.join(os.path.dirname(path), 'Data') 

BloombergData = pd.read_csv(data_path + "/BloombergData_Swap_Features.csv")
preData = pd.read_csv(data_path + "/TestData_Swap_Features_pre.csv")
postData = pd.read_csv(data_path + "/TestData_Swap_Features_post.csv")

insample = np.array(BloombergData.iloc[:,2:].reset_index(drop=True)) 
insample_scaled = [x/100 for x in insample]
insample_tensor = torch.from_numpy(np.float32(insample_scaled))

X_train1, X_val1 = train_validation_split(BloombergData.reset_index(drop=True), 0.1) 
X_train2  = np.array(X_train1.iloc[:,2:].reset_index(drop=True)) 
X_train_scaled = [x/100 for x in X_train2]
X_train_tensor = torch.from_numpy(np.float32(X_train_scaled))

X_val2  = np.array(X_val1.iloc[:,2:].reset_index(drop=True)) 
X_val_scaled = [x/100 for x in X_val2]
X_val_tensor = torch.from_numpy(np.float32(X_val_scaled))

preX_test2  = np.array(preData.iloc[:,2:].reset_index(drop=True)) 
preX_test_scaled = [x/100 for x in preX_test2]
preX_test_tensor = torch.from_numpy(np.float32(preX_test_scaled))

postX_test2  = np.array(postData.iloc[:,2:].reset_index(drop=True)) 
postX_test_scaled = [x/100 for x in postX_test2]
postX_test_tensor = torch.from_numpy(np.float32(postX_test_scaled))

In [ ]:
class nn_centered_softmax(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 1 / (1 + torch.exp(-x)) - 0.5
    
def grad_latent_2factor(res, model):

    calc_grad_G = vmap(jacfwd(model.decoder, argnums=0), in_dims=(0)) 
    grad_G = calc_grad_G(res).squeeze(dim=1) # N*30X3

    gradG_dz   = grad_G[:, :2] # N*30X2
    dG_dm      = grad_G[:, 2:] # N*30X1

    return gradG_dz, dG_dm

def hess_latent_2factor(res, model):
    
    calc_hess_G = vmap(jacfwd(jacfwd(model.decoder, argnums=0), argnums=0), in_dims=(0))
    hess_G = calc_hess_G(res).squeeze(dim=1)

    hessG_dz = hess_G[:,:2,:2]
    
    return hessG_dz

def alpha_fct(res, model, mu, sigma, G):
    gradG_dz, dG_dm = grad_latent_2factor(res, model)
    hessG_dz = hess_latent_2factor(res, model)

    part1 = -dG_dm

    part2 = torch.matmul(gradG_dz.unsqueeze(1), mu.unsqueeze(2))#.detach()
    part2 = part2.squeeze(-1)

    part0 = torch.matmul(sigma.transpose(1, 2), torch.matmul(hessG_dz, sigma))#.detach()

    part3 = 0.5 * torch.einsum('bii->b', part0).unsqueeze(-1)

    return (part1 + part2 + part3) / G

def beta_fct(r,G):
    return r/G

def gamma_fct(res, model, sigma):
    gradG_dz, _ = grad_latent_2factor(res, model)

    grad_grad = torch.matmul(gradG_dz.unsqueeze(-1), gradG_dz.unsqueeze(-1).transpose(1, 2))
    part0 = torch.matmul(sigma.transpose(1, 2), grad_grad.matmul(sigma))

    return 0.5 * torch.einsum('bii->b', part0).unsqueeze(-1)

def build_sigma_matrix_2factor(sigma_1, sigma_2, rho):
    sigma_matrix = torch.zeros((len(sigma_1), 2, 2))  
    sigma_matrix[:, 0, 0] = torch.exp(sigma_1.squeeze(-1))
    sigma_matrix[:, 0, 1] = 0  
    sigma_matrix[:, 1, 0] = torch.tanh(rho.squeeze(-1)) * torch.exp(sigma_2.squeeze(-1))
    sigma_matrix[:, 1, 1] = torch.sqrt(1 - torch.tanh(rho.squeeze(-1))**2) * torch.exp(sigma_2.squeeze(-1))

    return sigma_matrix.repeat_interleave(30, dim=0)

class ODESystem(torch.nn.Module):
    def __init__(self, alpha, beta, gamma):
        super(ODESystem, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma

    def forward(self, t, x):

        alpha_t = piecewise_constant_param(t, self.alpha)
        beta_t = piecewise_constant_param(t, self.beta)
        gamma_t = piecewise_constant_param(t, self.gamma)

        dxdt = torch.zeros_like(x)
        
        dxdt[:, 0] = gamma_t * x[:, 1]**2
        dxdt[:, 1] = alpha_t * x[:, 1] + beta_t
        
        return dxdt
    
def reshape_wide(x, length):
    x = x.reshape(length, 30)

    return x

def piecewise_constant_param(t, param_table):
    if not isinstance(t, torch.Tensor):
        t = torch.tensor(t, dtype=torch.float32, requires_grad=True)
    
    t_clamped = torch.clamp(t, min=0, max=30)

    idx = torch.ceil(t_clamped).detach() + t_clamped - t_clamped.detach() - 1
    idx = idx.long() 

    param1 = param_table[:, idx]

    return param1 

In [ ]:
def arb_equation_2factor(G, sigma_1, sigma_2, rho , mu, r, encoded_mat, model, dA, dB, B):
    N = len(G)
    
    r_long = r.repeat_interleave(30, dim=0) 
    G_long = G.reshape(N*30, 1)
    dA_long = dA.reshape(N*30, 1)
    dB_long = dB.reshape(N*30, 1)
    B_long = B.reshape(N*30, 1)
    mu_long = mu.repeat_interleave(30, dim=0)      
    grad_z, dy_dm   = grad_latent_2factor(encoded_mat, model)  
    
    
    grad_zmu = torch.matmul(grad_z.unsqueeze(1), mu_long.unsqueeze(2)).squeeze(-1) 
    
    hess_z  = hess_latent_2factor(encoded_mat, model) 
    sigma_long = build_sigma_matrix_2factor(sigma_1, sigma_2, rho)
    sigma_hess_sigma = torch.matmul(sigma_long.transpose(1, 2), torch.matmul(hess_z, sigma_long)) 
    trace_hess = 0.5 * torch.einsum('bii->b', sigma_hess_sigma).unsqueeze(-1)
    
    grad_grad = torch.matmul(grad_z.unsqueeze(-1), grad_z.unsqueeze(-1).transpose(1, 2))
    sigma_grad_grad_sigma = torch.matmul(sigma_long.transpose(1, 2), torch.matmul(grad_grad, sigma_long)) 
    trace_grad_grad = 0.5 * torch.einsum('bii->b', sigma_grad_grad_sigma).unsqueeze(-1)
    
    final_term = -r_long - dA_long + G_long*dB_long + B_long*(dy_dm - grad_zmu - trace_hess) + (B_long**2)*trace_grad_grad   
    
    return final_term

def reconstruct(G_output, sigma_1, sigma_2, rho, mu, r, encoded_mat, N, model):
    sigma_long = build_sigma_matrix_2factor(sigma_1, sigma_2, rho)  # all N*30 long
    mu_long = mu.repeat_interleave(30, dim=0)
    G_long = G_output.reshape(-1, 1)
    r_long = r.repeat_interleave(30, dim=0)
    
    
    alpha = reshape_wide(alpha_fct(encoded_mat, model, mu_long, sigma_long, G_long), N)
    beta = reshape_wide(beta_fct(r_long, G_long), N)
    gamma = reshape_wide(gamma_fct(encoded_mat, model, sigma_long), N)
    
    X_0 = torch.zeros(N, 2)

    t_span = torch.arange(0, 31, dtype=torch.float32) 
    ode_system = ODESystem(alpha, beta, gamma)
    dX_sol = odeint(ode_system, X_0, t_span, method='rk4')
    dX_sol = dX_sol.permute(1, 0, 2)
    A = dX_sol[:, 1:, 0] # shape (N,)
    B = dX_sol[:, 1:, 1]  # shape (N,)

    p = torch.exp(torch.clamp((A-B*G_output), min = -80, max = 80)) # N,30
    
    p_cumsum = torch.cumsum(p, axis=1) # N,30
    
    s = (1-p)/p_cumsum # N,30
    
    p_long = p.reshape(N*30,1)
    
    # Arbitrage loss
    dA = B**2 * gamma 
    dB = B * alpha + beta
   
    arb_l = arb_equation_2factor(G_output, sigma_1, sigma_2, rho, mu, r, encoded_mat, model, dA, dB, B)

    plot_all(alpha, beta, gamma, A, B)
    

    return s, arb_l

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(8, 2, bias = False)
        )

        self.sigma = nn.Sequential(                
            nn.Linear(2, 4, bias = False),
            nn_centered_softmax(), 
            nn.Linear(4, 3, bias = False)
        )

        self.mu = nn.Sequential(
            nn.Linear(2, 2, bias = True)
        )

        self.r = nn.Sequential(
            nn.Linear(2, 4, bias = False),
            nn_centered_softmax(),
            nn.Linear(4, 1, bias = False)    
        )

        self.decoder = nn.Sequential(
            nn.Linear(3, 10, bias = False),
            nn_centered_softmax(),
            nn.Linear(10, 1, bias = False)
        )

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                # if m.bias is not None: 
                #     nn.init.xavier_uniform_(m.bias) # maybe works?

    def forward(self, x):
        
        encoded = self.encoder(x) 
    
        mat = torch.arange(1, 31, dtype=torch.float32).repeat(len(encoded)) # length N*30 array of 1-30,1-30 and so on N times - range(1,31)*N
        x1 = encoded.repeat_interleave(30, dim=0)                           # N*30x2 
        encoded_mat = torch.cat([x1, mat.unsqueeze(1)], dim=1)              # N*30x3

        # Decoder --> gives y for all maturities 
        G_output = self.decoder(encoded_mat).reshape(len(x), 30)

        # K network for sigma 
        sigma_1, sigma_2, rho = torch.split(self.sigma(encoded), 1, dim=1)

        # H network for mu
        mu = self.mu(encoded)

        # R network for rate 
        r = self.r(encoded)

        return G_output, sigma_1, sigma_2, rho, mu, r, encoded_mat

In [ ]:
#torch.manual_seed(1)

# model = Autoencoder()   
# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.005)
# scheduler = LR_Scheduler(optimizer, percentage=0.9, interval = 50)


# # Training
# num_epochs = 3496
# batch_size = 32
# data_loader = torch.utils.data.DataLoader(X_train_tensor, batch_size=batch_size, shuffle=True)

# swap_mats = [1, 2, 3, 5, 10, 15, 20, 30]
# swap_mats0 = [i-1 for i in swap_mats]
# arb_losses_list = []
# losses_list = []
# vallosses_list = []
# for epoch in range(num_epochs):
#     arb_loss_epoch = 0
#     loss_epoch = 0
#     for batch in data_loader:

#         N = len(batch)

#         G_output, sigma_1, sigma_2, rho, mu, r, encoded_mat = model(batch)

#         reconstructed, arb_loss = reconstruct(G_output, sigma_1, sigma_2, rho, mu, r, encoded_mat, N, model)
#         #print(reconstructed.shape)
#         s_final = reconstructed[:, swap_mats0]
#         #print(s_final)
#         loss = criterion(s_final, batch)
        
#         # Backward pass and optimization

#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
#         optimizer.zero_grad()
#         loss.backward(retain_graph=True)

#         nan_detected = False
#         for param in model.parameters():
#             if param.grad is not None and torch.isnan(param.grad).any():
#                     print(f"NaN detected in gradients at epoch {epoch}, resetting to zero.")
#                     param.grad.zero_() 
#                     nan_detected = True

#         if not nan_detected:
#                 optimizer.step()
#         else:
#                 print(f"Skipping optimizer update at epoch {epoch} due to NaNs.")

#         #arb_losses_list.append(arb_loss)
#         arb_loss_epoch += torch.sum(arb_loss)
#         loss_epoch += loss

#      #
#     N = len(X_val_tensor)    
#     G_output, sigma_1, sigma_2, rho, mu, r, encoded_mat = model(X_val_tensor)
#     reconstructed, arb_loss  = reconstruct(G_output, sigma_1, sigma_2, rho, mu, r, encoded_mat, N, model)
#     s_final = reconstructed[:, swap_mats0]
#     loss_val = criterion(s_final, X_val_tensor)

#     #
#     N = len(X_train_tensor)    
#     G_output, sigma_1, sigma_2, rho, mu, r, encoded_mat = model(X_train_tensor)
#     reconstructed, arb_loss  = reconstruct(G_output, sigma_1, sigma_2, rho, mu, r, encoded_mat, N, model)
#     s_final = reconstructed[:, swap_mats0]
#     loss_train = criterion(s_final, X_train_tensor)

#     # append relevant data
#     vallosses_list.append(loss_val.item())
#     arb_losses_list.append(arb_loss_epoch.item())
#     losses_list.append(loss_train.item())

#     scheduler.step()

#     if (epoch + 1) % 100 == 0:
#         print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss_train.item():.8f}, Arbitrage loss: {(arb_loss_epoch)}")

In [ ]:
# # os.makedirs("models", exist_ok=True)
name = "2factor_AF_odeint"
# torch.save(model.state_dict(), f"models/{name}.pth")
# print(f"Model saved successfully as models/{name}.pth")
model2 = Autoencoder()       # Ensure Autoencoder class is defined
model2.load_state_dict(torch.load(f"models/{name}.pth"))